<a href="https://colab.research.google.com/github/Jang-KyungWuk/KR_Community_Word_Cloud_Generator/blob/master/Keyword%20Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
<current version 1.2>
Issue that DC article crawling is not working properly has been fixed
Keyword Analyzer now use konlpy version 0.5.1 (legacy version)
Emoji and more than 1 blank is now deleted BEFORE morpheme analyze

<future version 1.3 will try>
Test Ruliweb crawling and fix if there're bugs
Add word2vec feature (can be postponded)
'''

"\n<current version 1.2>\nIssue that DC article crawling is not working properly has been fixed\n\n<future version 1.3 will try>\nTest Ruliweb crawling and fix if there're bugs\nAdd word2vec feature (can be postponded)\n"

In [2]:
def setup():
  '''
  FOR GOOGLE COLAB USERS START setup() 
  AND RESTART RUNTIME (CTRL+M) 
  THEN START setup() TO CHECK WHETHER KR FONT IS PRINTED CORRECTLY
  '''
  
  # Mount Google Drive
  from google.colab import drive
  drive.mount('/content/gdrive')
  base_dir="/content/gdrive/My Drive/"

  # KoNLPy Install
  !apt-get update
  !apt-get install g++ openjdk-8-jdk 
  !pip3 install konlpy==0.5.1

  # Install KR Font
  !apt -qq -y install fonts-nanum
  fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
  KR_Font_initializer(fontpath)

  # Install emoji to delete emojis
  !pip install emoji

  return base_dir

def KR_Font_initializer(fontpath):
  # For draw legend
  import matplotlib.pyplot as plt
  import matplotlib as mpl
  import matplotlib.font_manager as fm

  # Set KR Font for matplotlib
  font=fm.FontProperties(fname=fontpath,size=20)
  plt.rc('font',family='NanumBarunGothic')
  mpl.font_manager._rebuild()

  plt.bar(['KR font','성공'],[1,2])
  plt.show()

In [3]:
def get_current_time_code():
  '''time_code()
  Generate time code from current time
  '''

  # Get current time (KST = GMT +9)
  KST=datetime.timezone(datetime.timedelta(hours=9))
  now=datetime.datetime.now(KST)
  MM=str(now.month)
  DD=str(now.day)
  hh=str(now.hour)
  mm=str(now.minute)
  ss=str(now.second)
  c_time=[MM,DD,hh,mm,ss]
  for cursor in range (len(c_time)):
    if len(c_time[cursor])==1:
      c_time[cursor]="0"+c_time[cursor]
  time_code='-'.join(c_time)

  return time_code

In [4]:
# Function for coloring
def color_func(word, font_size, position, orientation, random_state=None, **kwargs):
  return tuple(Dark2_8.colors[random.randint(0,7)])

In [5]:
# Function to find titles at DCinside
def dc_title_finder(tag):
  return tag.has_attr('href') and not tag.has_attr('class') and not tag.has_attr('rel')

In [6]:
def dcinside_crawl(pages,txt_files,crawl_title,crawl_article,
                   is_m_gallery,board_id,
                   request_delay,
                   dc_title_tag_pos, dc_official_announce,
                   user_agent,
                   base_dir):

  main_url="https://gall.dcinside.com"
  base_url=main_url*1
  if is_m_gallery==True:
    base_url+="/mgallery"
  base_url+=(("/board/lists/?id=%s&page=")%board_id)

  title_tag_pos_min, title_tag_pos_max=dc_title_tag_pos[0]+dc_official_announce, dc_title_tag_pos[1]+dc_official_announce

  for crawl_sector_cursor in range (len(pages)):
    crawl_min,crawl_max=pages[crawl_sector_cursor]

    if crawl_title==True:
      title_crawl_txt=open(base_dir + txt_files[0][crawl_sector_cursor],'w')
      
    if crawl_article==True:
      link_crawl_txt=open(base_dir + txt_files[1][crawl_sector_cursor],'w')


    for page in range (crawl_min,crawl_max):
      url=base_url+str(page)
      html=rq.get(url, headers={'User-Agent': user_agent}).text
      soup=BeautifulSoup(html,'lxml')
      tags=soup.find_all(dc_title_finder)
      bbs_tags=tags[title_tag_pos_min:title_tag_pos_max]
      for t in bbs_tags:
        if crawl_title==True:
          title_crawl_txt.write(t.text+'\n')
        if crawl_article==True:
          link_crawl_txt.write(main_url+t['href']+'\n')
      time.sleep(request_delay)

    if crawl_title==True:
      title_crawl_txt.close()

    if crawl_article==True:
      link_crawl_txt.close()

  if crawl_article==True:
    for crawl_sector_cursor in range (len(pages)):
      dirs=np.loadtxt(base_dir+txt_files[1][crawl_sector_cursor],delimiter="\n", dtype="str")
      article_crawl_txt=open(base_dir+txt_files[2][crawl_sector_cursor],'w')
      for dir in dirs:
        html=rq.get(dir, headers={'User-Agent':'el anemos keyword analyzer for word cloud'}).text
        soup=BeautifulSoup(html,'lxml')
        article=soup.body.find('div',{'style':'overflow:hidden;width:900px'}).text
        article=article.replace('\n',' ').replace('\u200b',' ').replace('\xa0','').replace('- dc official App','')
        article_crawl_txt.write(article+'\n')
        time.sleep(request_delay)
      
      article_crawl_txt.close()

In [7]:
def ruliweb_crawl(pages,txt_files,crawl_title,crawl_article,board_id,request_delay,user_agent,base_dir):

  # For web crawling
  from bs4 import BeautifulSoup
  import requests as rq
  import re
  
  main_url="https://bbs.ruliweb.com/community/board"
  base_url=main_url*1
  base_url+=(("/%s?page=")%board_id)

  for crawl_sector_cursor in range (len(pages)):
    crawl_min,crawl_max=pages[crawl_sector_cursor]

    if crawl_title==True:
      title_crawl_txt=open(base_dir + txt_files[0][crawl_sector_cursor],'w')
      
    if crawl_article==True:
      link_crawl_txt=open(base_dir + txt_files[1][crawl_sector_cursor],'w')


    for page in range (crawl_min,crawl_max):
      url=base_url+str(page)
      html=rq.get(url, headers={'User-Agent': user_agent}).text
      soup=BeautifulSoup(html,'lxml')
      title_soup=soup.find_all('a', attrs={'class':'deco'})
      title_tags=title_soup[len(title_soup)-32 : len(title_soup)-4]

      for t in bbs_tags:
        if crawl_title==True:
          title_crawl_txt.write(t.text+'\n')
        if crawl_article==True:
          link_crawl_txt.write(main_url+t['href']+'\n')
      time.sleep(request_delay)

    if crawl_title==True:
      title_crawl_txt.close()

    if crawl_article==True:
      link_crawl_txt.close()

  if crawl_article==True:
    for crawl_sector_cursor in range (len(pages)):
      dirs=np.loadtxt(base_dir+txt_files[1][crawl_sector_cursor],delimiter="\n", dtype="str")
      article_crawl_txt=open(base_dir+txt_files[2][crawl_sector_cursor],'w')
      for dir in dirs:
        html=rq.get(dir, headers={'User-Agent':'el anemos keyword analyzer for word cloud'}).text
        soup=BeautifulSoup(html,'lxml')
        article_tags=soup.find_all("div",class_="view_content")
        article_text=''
        for article_tag in article_tags:
          article_text+=(article_tag.text+' ')
        article_text=article_text.replace('\u200b',' ')
        article_crawl_txt.write(article_text+'\n')
        
        time.sleep(request_delay)
      
      article_crawl_txt.close()

In [8]:
def name_txt_files(site,board_id,pages,time_code, crawl_title, crawl_article):
  txt_files=[[],[],[]] #Title, link, article

  if crawl_title==True:
    for crawl_page in pages:
      crawl_min,crawl_max=crawl_page
      txt_files[0].append(("%s %s title crawl %s %s~%s.txt")%(site, board_id, time_code, crawl_min, crawl_max-1))

  if crawl_article==True:
    for crawl_page in pages:
      crawl_min,crawl_max=crawl_page
      txt_files[1].append(("%s %s link crawl %s %s~%s.txt")%(site, board_id, time_code, crawl_min, crawl_max-1))
      txt_files[2].append(("%s %s article crawl %s %s~%s.txt")%(site, board_id, time_code, crawl_min, crawl_max-1))
  
  return txt_files

In [9]:
def wordseed_reader(file_names,crawl_title,crawl_article,base_dir,pages):
  wordseeds=[]
  len_pages=len(pages)

  for crawl_sector_cursor in range (len_pages):
    wordseed=""
    if crawl_title==True:
        title_crawl_txt=open(base_dir+file_names[0][crawl_sector_cursor])
        wordseed+=title_crawl_txt.read()
        title_crawl_txt.close()

    if crawl_article==True:
        article_crawl_txt=open(base_dir+file_names[2][crawl_sector_cursor])
        wordseed+=article_crawl_txt.read()
        article_crawl_txt.close()
    wordseed=blank_emoji_deleter(wordseed)
    wordseeds.append(wordseed)

  return wordseeds

In [10]:
def blank_emoji_deleter(wordseed):
  # Delete '\n (Enter key input)'
  wordseed=wordseed.replace('\s+',' ').replace('\n',' ').replace('\u3000',' ')

  # Delete Emoji
  wordseed=emoji.get_emoji_regexp().sub(u'',wordseed)

  return wordseed

In [11]:
def morpheme_analysis(wordseed,user_dic,base_dir):
  tagged_words=konlpy.tag.Komoran(userdic=base_dir+user_dic).pos(wordseed)
  grammar="""
NP: {<N.*>*<Suffix>?}   # Noun phrase
VP: {<V.*>*}            # Verb phrase
AP: {<A.*>*}            # Adjective phrase
"""
  parser=nltk.RegexpParser(grammar)

  return tagged_words

In [12]:
def morpheme_tag_filtter(tagged_words,selected_tags):
  filtered_word_seed=""
  for morpheme in tagged_words:
    if morpheme[1] in selected_tags:
      filtered_word_seed+=("%s "%morpheme[0]) 
  return filtered_word_seed

In [13]:
def keyword_ranking(wordseed,show_rank,board_id,time_code,crawl_page,base_dir,site):

  crawl_min, crawl_max=crawl_page

  wordseed_list=wordseed.split()
  keywords=[]
  counts=[]

  for keyword in wordseed_list:
    if keyword in keywords:
      counts[keywords.index(keyword)]+=1
    else:
      keywords.append(keyword)
      counts.append(1)

  plt_kw=[]
  plt_ct=[]

  for rank_cursor in range (show_rank):
    index=counts.index(max(counts))
    plt_kw.append(keywords.pop(index))
    plt_ct.append(counts.pop(index))
  plt_kw.reverse()
  plt_ct.reverse()
  plt.barh(plt_kw,plt_ct)
  plt.title("상위 %s개 키워드 등장 횟수"%(show_rank))
  plt.xlabel("등장 횟수")
  plt.savefig(base_dir+("%s %s Keyword Ranking %s %s~%s.png"%(site,board_id,time_code,crawl_min,crawl_max-1)))
  plt.show()

In [14]:
def wordcloud_generator(wordseed,mask_image, site, board_id, time_code, crawl_page,fontpath,base_dir):

  crawl_min, crawl_max=crawl_page

  # Word cloud generate (Komoran+user_dic)
  icon=Image.open(base_dir+mask_image)
  mask=Image.new("RGB",icon.size, (255,255,255))
  mask.paste(icon,icon)
  mask=np.array(mask)

  wc=WordCloud(font_path=fontpath, background_color="white",max_words=150,mask=mask,max_font_size=400,random_state=1234567)

  wc.generate_from_text(wordseed)
  wc.recolor(color_func=color_func, random_state=1234567)
    
  wc_dir=base_dir+("%s %s Word Cloud %s %s~%s.png")%(site, board_id,time_code,crawl_min,crawl_max-1)
  wc.to_file(wc_dir)

In [15]:
def Keyword_analyzer(site,board_id,crawl_pages,cloud_mask,crawl_title=True, crawl_article=False,
                     show_ranking=True,show_rank=5, is_m_gallery=False,
                     custom_word_corpus="",base_dir="", request_delay=1,
                     dc_title_tag_pos=[21,71],dc_official_announce=0,
                     user_agent="'el anemos keyword analyzer for word cloud | contact elixir95@naver.com",
                     random_seed='1234567',fontpath='/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf',
                     selected_tags=["NNG","NNP","NR","IC","SL","SH","SW","SN"]):
  '''Keyword_analyzer(site,pages,board_id,crawl_title=True, crawl_article_False, is_m_gallery=False)
  All in one function that do webcrawl, create wordcloud, legend
  site supports dcinside, ruliweb
  '''

  time_code=get_current_time_code()
  txt_files=name_txt_files(site,board_id,crawl_pages,time_code, crawl_title, crawl_article)

  if site in (["DCinside","DC","dc","dcinside","dcinside.com"]):
    dcinside_crawl(crawl_pages,txt_files,crawl_title,crawl_article,
                   is_m_gallery,board_id,
                   request_delay,
                   dc_title_tag_pos, dc_official_announce,
                   user_agent,
                   base_dir)

  elif site in (["Ruliweb","ruliweb","ruliweb.com"]):
    ruliweb_crawl(crawl_pages,txt_files,crawl_title,crawl_article,board_id,request_delay,user_agent,base_dir)

  wordseeds=wordseed_reader(file_names=txt_files,crawl_title=crawl_title,crawl_article=crawl_article,base_dir=base_dir,pages=crawl_pages)


  for file_cursor in range(len(crawl_pages)):

    tagged_wordseed=morpheme_analysis(wordseeds[file_cursor],custom_word_corpus,base_dir)
    filttered_wordseed=morpheme_tag_filtter(tagged_wordseed,selected_tags)

    if show_ranking==True:
      keyword_ranking(wordseed=filttered_wordseed,show_rank=show_rank,board_id=board_id,time_code=time_code,crawl_page=crawl_pages[file_cursor],base_dir=base_dir, site=site)
    
    wordcloud_generator(wordseed=filttered_wordseed,mask_image=cloud_mask, site=site, board_id=board_id, time_code=time_code,crawl_page=crawl_pages[file_cursor],fontpath=fontpath,base_dir=base_dir)

In [ ]:
# Download libraries
base_dir=setup()

# Import libraries

# For web crawling
from bs4 import BeautifulSoup
import requests as rq
import re

# For naming
import datetime

# For word cloud generating
import numpy as np
import random
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
from palettable.colorbrewer.qualitative import Dark2_8

# For kr analysis
import konlpy
import nltk

# For set delay
import time

# For emoji delete
import emoji

# For draw legend
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

In [ ]:
Keyword_analyzer(site='dc',
                 board_id='hypergryph',
                 crawl_pages=[[2,3]],
                 crawl_title=False, 
                 crawl_article=True,
                 show_ranking=True,
                 show_rank=10,
                 is_m_gallery=True,
                 cloud_mask='blemishine_b.png',
                 custom_word_corpus="dc_ark_user_dic_v20201018.txt",
                 dc_official_announce=1,
                 base_dir=base_dir)